In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

In [ ]:
year    = '10'
quarter = '4'

In [ ]:
file_name_origdata_csv = 'origdata 20' + year + '-Q' + quarter + '.csv'
file_name_MP_csv       = 'svcgdata 20' + year + '-Q' + quarter + '.csv'
Orig = pd.read_csv(file_name_origdata_csv)
MP   = pd.read_csv(file_name_MP_csv)

In [ ]:
MP0 = pd.DataFrame(MP)
MP0.drop_duplicates(subset = "id_loan", keep = 'first', inplace = True)
MP0.reset_index(drop = True, inplace = True)
MP0['svcg_cycle'] = MP0['svcg_cycle'].astype(str)
MP0['svcg_cycle'] = MP0['svcg_cycle'].apply(lambda x: x + '01')
MP0['dt_orig_0']  = (MP0['svcg_cycle'].apply(lambda x: datetime(year = int(x[0:4]),
                                                                month = int(x[4:6]), day = int(x[6:8]))))

def f(x, y):
    return (x - relativedelta(months = y + 1)).strftime("%Y%m");

MP0['dt_orig'] = MP0.apply(lambda x: f(x['dt_orig_0'], x['loan_age']), axis = 1)
MP0['dt_orig'] = MP0['dt_orig'].astype(int)
drop_list = [column for column in MP0.columns if column not in ['id_loan', 'dt_orig']]
MP0.drop(drop_list, axis = 1, inplace = True)

In [ ]:
MP.drop(MP[MP['cd_zero_bal'] == 15].index, inplace = True)
MP.drop(MP[MP['delq_sts'] == 'XX'].index, inplace = True)
MP['delq_sts'] = MP['delq_sts'].where(MP['delq_sts'] != 'R', '-1')
MP['delq_sts'] = MP['delq_sts'].astype(int)
defaults = (MP.loc[(MP['delq_sts'] == -1)   | (MP['delq_sts'] >= 3) | (MP['cd_zero_bal'] == 3) |
                   (MP['cd_zero_bal'] == 6) | (MP['cd_zero_bal'] == 9)])
MP['delq_sts'] = MP['delq_sts'].astype(str)
defaults['delq_sts'] = defaults['delq_sts'].astype(str)
MP['delq_sts'] = MP['delq_sts'].where(MP['delq_sts'] != '-1', 'R')
defaults['delq_sts'] = defaults['delq_sts'].where(defaults['delq_sts'] != '-1', 'R')
defaults.drop_duplicates(subset = "id_loan", keep = 'first', inplace = True)
defaults.reset_index(drop = True, inplace = True)
defaults['Default'] = 'YES'
drop_list = [column for column in defaults.columns if column not in ['id_loan', 'svcg_cycle', 'Default']]
defaults.drop(drop_list, axis = 1, inplace = True)

In [ ]:
Orig1 = pd.DataFrame(Orig)
Orig1 = Orig1[Orig1['id_loan'].isin(list(MP0['id_loan']))]
Orig1.reset_index(drop = True, inplace = True)
Orig2 = pd.merge(Orig1, MP0, on = 'id_loan', how = 'outer')
mergedDf = pd.merge(Orig2, defaults, on = 'id_loan', how = 'outer')
mergedDf['svcg_cycle'] = mergedDf['svcg_cycle'].where(mergedDf['svcg_cycle'].isnull() == False, 'N_A')
mergedDf['svcg_cycle'] = mergedDf['svcg_cycle'].astype(str)
mergedDf['Default'] = mergedDf['Default'].where(mergedDf['Default'].isnull() == False, 'NO')
mergedDf.drop(['Unnamed: 0'], axis=1, inplace = True)
cols = mergedDf.columns
cols = cols.drop('id_loan')
cols = ['id_loan'] + list(cols)
mergedDf = mergedDf[cols]

In [ ]:
Data = mergedDf.copy()
Data['fico'].loc[Data['fico'] == 9999] = np.nan
Data['flag_fthb'].loc[Data['flag_fthb'] == '9'] = np.nan
Data['mi_pct'].loc[Data['mi_pct'] == 999] = np.nan
Data['cnt_units'].loc[Data['cnt_units'] == 99] = np.nan
Data['occpy_sts'].loc[Data['occpy_sts'] == '9'] = np.nan
Data['cltv'].loc[Data['cltv'] == 999] = np.nan
Data['dti'].loc[Data['dti'] == 999] = np.nan
Data['ltv'].loc[Data['ltv'] == 999] = np.nan
Data['channel'].loc[Data['channel'] == '9'] = np.nan
Data['prop_type'].loc[Data['prop_type'] == '99'] = np.nan
Data['loan_purpose'].loc[Data['loan_purpose'] == '9'] = np.nan
Data['cnt_borr'].loc[Data['cnt_borr'] == 99] = np.nan
Data['flag_sc'].fillna('N', inplace = True)
Data['flag_fthb'].fillna('Other', inplace = True)

In [ ]:
n = len(Data.index)
for column in Data:
    null = Data[column].isnull().sum()

    if (null / n) > 0.8:
        Data = Data.drop(column, axis = 1)

    if Data[column].dtype == 'object':
        Data[column].fillna(Data[column].mode()[0], inplace = True)
    else:
        Data[column].fillna(Data[column].median(), inplace = True)

In [ ]:
Data['year'] = Data['dt_orig'].astype(str).apply(lambda x: x[0:4]).astype(int)
Data['month'] = Data['dt_orig'].astype(str).apply(lambda x: x[4:6]).astype(int)
Data.reset_index(drop = True, inplace = True)
Data.drop(['dt_orig'], axis = 1, inplace = True)

In [ ]:
MP = MP[MP['id_loan'].isin(list(Data['id_loan']))]
MP.reset_index(drop = True, inplace = True)
drop_list = [column for column in MP.columns if column not in ['id_loan', 'svcg_cycle', 'current_upb',
                                                               'delq_sts', 'loan_age', 'mths_remng', 'current_int_rt']]
MP.drop(drop_list, axis = 1, inplace = True)

In [ ]:
l1 = list(set(MP[MP['loan_age'] == 0]['id_loan'].unique()).intersection(set(MP[MP['loan_age'] == 60]['id_loan'].unique())))
Data = Data[Data['id_loan'].isin(l1)]
Data.reset_index(drop = True, inplace = True)
MP = MP[MP['id_loan'].isin(l1)]
MP.reset_index(drop = True, inplace = True)

In [ ]:
Data.to_csv('df_orig_' + year + '_' + quarter + '.csv', index = False)
MP.to_csv('df_mp_' + year + '_' + quarter + '.csv', index = False)